# Yet Another OpenPose 

---
## Environment

In [50]:
#verify user
!whoami
#verify user account, if running on google cloud, otherwise ignore
!gcloud auth list
#which Environment/virtualenv running in
import sys
import os

print("Python running from:",sys.prefix)
print("Current working dir",os.getcwd())

pop_computer\sirawit
Python running from: C:\Users\ASUS\AppData\Local\Programs\Python\Python38
Current working dir c:\Users\ASUS\Desktop\model1\training


'gcloud' is not recognized as an internal or external command,
operable program or batch file.


In [51]:
#start tensor board
# must run 
#/usr/local/bin/tensorboard serve --logdir gs://dl_training_results/tensorboard --port 8889 --bind_all

In [52]:
#start TPU
!gcloud compute tpus start node-1 --zone us-central1-c
#stop TPU
#!gcloud compute tpus stop node-1 --zone us-central1-c

'gcloud' is not recognized as an internal or external command,
operable program or batch file.


---
## Config

In [53]:
import sys
sys.path.append("..")

import configs.default_config as cfg

import configs.local_storage_config as storage_cfg
#import remote_storage_config as storage_cfg
cfg.__dict__.update(storage_cfg.__dict__)

#import tpu_training.TPU_config as TPU_config
#cfg.__dict__.update(TPU_config.__dict__) #comment out to disable TPU training

cfg.RUN_NAME="model11" #for reference 

In [54]:
from utils import now
nowt=now()

In [55]:
import tensorflow as tf
print("Tensorflow version:",tf.version.VERSION)

Tensorflow version: 2.8.0


In [56]:
if cfg.TPU_MODE:
    import tpu_training.init_TPU as init_TPU
    
    print("Testing results bucket connectivity")
    !touch /tmp/test
    !gsutil cp /tmp/test {cfg.TENSORBOARD_PATH}/test
    !gsutil rm {cfg.TENSORBOARD_PATH}/test
    !gsutil cp /tmp/test {cfg.CHECKPOINTS_PATH}/test
    !gsutil rm {cfg.CHECKPOINTS_PATH}/test
    print("Testing dataset bucket connectivity")
    !gsutil ls gs://{cfg.GCS_TFRECORDS_BUCKETNAME} | head -4
    print("Testing TPU connectivity")
    !nmap -Pn -p8470 {cfg.TPU_IP}
    strategy,resolver=init_TPU.init_tpu(cfg.TPU_IP) #This must be run before any imports!!!!

In [57]:
import datetime
import numpy as np

import dataset_functions
import models.six_stage_linear_model as model
import callbacks
import dataset_builder
import load_weights
import loss_metrics

In [58]:
# import importlib as il
# il.reload() #useful to reload any changes

---
## Dataset

In [59]:
# Training settings
TRAINING_EPOCHS = 30
REAL_EPOCH_STEPS = int(cfg.DATASET_SIZE / cfg.BATCH_SIZE)
SHORT_EPOCH_STEPS=50 #actual epocsh used in training, smaller than real epoch, but allows to track progress better, [in batches]
SHORT_TRAINING_EPOCHS=int(TRAINING_EPOCHS*(REAL_EPOCH_STEPS/SHORT_EPOCH_STEPS))
SHORT_VALIDATION_STEPS=5 #per short epoch

EPOCH_RATIO=int(REAL_EPOCH_STEPS / SHORT_EPOCH_STEPS)
# adam_learning_rate=0.001  #for reference
BASE_LEARNING_RATE = 0.001
LEARNING_RATE_SCHEDUELE = np.zeros(100000)  #used with short epochs
LEARNING_RATE_SCHEDUELE[:3 * EPOCH_RATIO] = 1
LEARNING_RATE_SCHEDUELE[3 * EPOCH_RATIO:20 * EPOCH_RATIO] = 1
LEARNING_RATE_SCHEDUELE[20 * EPOCH_RATIO:40 * EPOCH_RATIO] = 1
LEARNING_RATE_SCHEDUELE[40 * EPOCH_RATIO:100 * EPOCH_RATIO] = 0.5
LEARNING_RATE_SCHEDUELE[100 * EPOCH_RATIO:] = 0.3
LEARNING_RATE_SCHEDUELE *= BASE_LEARNING_RATE

In [60]:
model_ds=model.ModelDatasetComponent(cfg)

tfrecord_files_train=dataset_builder.get_tfrecord_filenames(cfg.TRAIN_TFRECORDS,cfg)
tfrecord_files_valid=dataset_builder.get_tfrecord_filenames(cfg.VALID_TFRECORDS,cfg)
print("Found the following training TFrecords:\n","\n".join(tfrecord_files_train))
print("Found the following validation TFrecords:\n","\n".join(tfrecord_files_valid))

print("Building training dataset")
dst=dataset_builder.build_training_ds(tfrecord_files_train,model_ds.place_training_labels,cfg)
print("Training dataset shape:",dst)
print("Building validation dataset")
dsv=dataset_builder.build_validation_ds(tfrecord_files_valid,model_ds.place_training_labels,cfg)
print("Validation dataset shape:",dsv)


Retrieving TFrecords from: ..\configs\..\dataset\TFrecords\training
Retrieving TFrecords from: ..\configs\..\dataset\TFrecords\validation
Found the following training TFrecords:
 ..\configs\..\dataset\TFrecords\training-001.tfrecords
..\configs\..\dataset\TFrecords\training-002.tfrecords
..\configs\..\dataset\TFrecords\training-003.tfrecords
..\configs\..\dataset\TFrecords\training-004.tfrecords
..\configs\..\dataset\TFrecords\training-005.tfrecords
..\configs\..\dataset\TFrecords\training-006.tfrecords
..\configs\..\dataset\TFrecords\training-007.tfrecords
..\configs\..\dataset\TFrecords\training-008.tfrecords
..\configs\..\dataset\TFrecords\training-009.tfrecords
..\configs\..\dataset\TFrecords\training-010.tfrecords
..\configs\..\dataset\TFrecords\training-011.tfrecords
..\configs\..\dataset\TFrecords\training-012.tfrecords
..\configs\..\dataset\TFrecords\training-013.tfrecords
..\configs\..\dataset\TFrecords\training-014.tfrecords
..\configs\..\dataset\TFrecords\training-015.tfreco

In [61]:
#Test element
# dst_iter=iter(dst)
# sample_elem=next(dst_iter)
# print("Dataset shape:",dst) #this should match the model input, and output stages

In [62]:
#testing mask
# sample_elem=next(dst_iter)
# m=sample_elem[1][1][0,...,0]
# import matplotlib.pyplot as plt
# plt.imshow(m)
# plt.colorbar()

---
## Model
### Load weights

In [63]:
if cfg.ASK_FOR_CHECKPOINTS:
    checkpoint,starting_epoch=load_weights.checkpoints_prompt(cfg)
else:
    checkpoint=None
    starting_epoch=0

Found no checkpoints


### Build Model

In [64]:
model_maker=model.ModelMaker(cfg) #must be outside scope to keep the graph clean
tf.keras.backend.clear_session() #to clean to backaend from the imported model
def define():
    train_model,test_model=model_maker.create_models()
    
#     if cfg.INCLUDE_MASK:
#         losses=[loss_metrics.MaskedMeanAbsoluteError()
#                 ,loss_metrics.MaskedMeanAbsoluteError()
#                 ,loss_metrics.MaskedMeanAbsoluteError()
#                 ,loss_metrics.MaskedMeanAbsoluteError()
#                 ,loss_metrics.MaskedMeanSquaredError()
#                 ,loss_metrics.MaskedMeanSquaredError()]                      
#     else:
#         raise NotImplementedError       
    
    #this must match the model output order
    metrics=[
              [loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]
             ,[loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]
             ,[loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]
             ,[loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]
             ,[loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]    
             ,[loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]
        ]
    
    train_model.compile(
                    optimizer=tf.keras.optimizers.Adam(BASE_LEARNING_RATE)                   
                    ,loss=loss_metrics.MaskedMeanSquaredError()
                    ,loss_weights=[5,5,5,5,1,1]
                    ,metrics=metrics                           
                   )
    return train_model,test_model

if cfg.TPU_MODE:
    with strategy.scope():
        train_model,test_model=define()
        if (checkpoint):
            train_model.load_weights(checkpoint)
else:
    train_model,test_model=define()
    if (checkpoint):
        train_model.load_weights(checkpoint)

### Callbacks

In [65]:
all_callbacks=[
    callbacks.make_LRscheduler_callback(LEARNING_RATE_SCHEDUELE)
    ,callbacks.PrintLR()
    ,tf.keras.callbacks.TerminateOnNaN()
]
if cfg.SAVE_CHECKPOINTS:
    #checkpoint_callback,checkpoint_path=callbacks.make_checkpoint_callback(cfg,nowt,REAL_EPOCH_STEPS*cfg.BATCH_SIZE)
    checkpoint_callback=callbacks.make_checkpoint_callback(cfg,nowt,REAL_EPOCH_STEPS*cfg.BATCH_SIZE)
    all_callbacks.append(checkpoint_callback)
if cfg.SAVE_TENSORBOARD:
    #tensorboard_callback,tensorboard_path=callbacks.make_tensorboard_callback(cfg,nowt,EPOCH_RATIO)
    tensorboard_callback=callbacks.make_tensorboard_callback(cfg,nowt,EPOCH_RATIO)
    all_callbacks+=[tensorboard_callback]

In [66]:
#run to clean all tensorboard&checkpoints results
#!gsutil -m rm -r {cfg.TENSORBOARD_PATH}/*
#!gsutil -m rm -r {cfg.CHECKPOINTS_PATH}/*


---
## Training
Actually training

In [67]:
train_history=train_model.fit(
    dst
    ,epochs=SHORT_TRAINING_EPOCHS
    ,steps_per_epoch=SHORT_EPOCH_STEPS
    ,validation_steps=SHORT_VALIDATION_STEPS
    ,validation_data=dsv
    ,callbacks=all_callbacks
    ,initial_epoch=starting_epoch
)


Learning rate for epoch 0 is 0.0010000000474974513
Epoch 1/16800
50/50 [==============================] - 46s 590ms/step - loss: 0.3071 - s1pafs_output_loss: 0.0082 - s2pafs_output_loss: 0.0200 - s3pafs_output_loss: 0.0126 - s4pafs_output_loss: 0.0114 - s5kpts_output_loss: 0.0202 - s6kpts_output_loss: 0.0258 - s1pafs_output_MeanAbsoluteRatio: 1.2546 - s1pafs_output_analog_recall: 0.7807 - s2pafs_output_MeanAbsoluteRatio: 1.3597 - s2pafs_output_analog_recall: 0.7800 - s3pafs_output_MeanAbsoluteRatio: 1.2777 - s3pafs_output_analog_recall: 0.7797 - s4pafs_output_MeanAbsoluteRatio: 1.2710 - s4pafs_output_analog_recall: 0.7799 - s5kpts_output_MeanAbsoluteRatio: 1.0617 - s5kpts_output_analog_recall: 0.6750 - s6kpts_output_MeanAbsoluteRatio: 1.1176 - s6kpts_output_analog_recall: 0.6761 - val_loss: 0.1662 - val_s1pafs_output_loss: 0.0062 - val_s2pafs_output_loss: 0.0062 - val_s3pafs_output_loss: 0.0062 - val_s4pafs_output_loss: 0.0062 - val_s5kpts_output_loss: 0.0208 - val_s6kpts_output_loss:

KeyboardInterrupt: 

In [ ]:
#run to clear this run results
#!gsutil -m rm -r {tensorboard_path}/*
#!gsutil -m rm -r {checkpoint_path}/*

---
### Save results

In [ ]:
save_folder_name = "train_" +cfg.RUN_NAME+nowt
train_model.save_weights(os.path.join(cfg.MODELS_PATH ,save_folder_name,"weight")) 
json_config = train_model.to_json()
with open(os.path.join(cfg.MODELS_PATH,save_folder_name, "config.json"), "w") as json:
  json.write(json_config)

---
### Shutdown

In [ ]:
#shut down TPU
!gcloud compute tpus stop node-1 --zone us-central1-c

In [ ]:
#shut down this host
!gcloud compute instances stop instance-1 --zone us-central1-c

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_history.history['loss'])
plt.plot(train_history.history['val_loss'])
plt.title('model loss') 
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'], loc='upper left')
plt.show()

In [ ]:
json_file = open("../tmp/models/train_model1102Mon0522-0146/config.json", mode="r").read();
load_train_model = tf.keras.models.model_from_json(json_file);
load_train_model.load_weights("../tmp/models/train_model1102Mon0522-0146/weight");


In [ ]:
#train_model.save_weights(os.path.join(cfg.MODELS_PATH ,save_folder_name,"weight.h5")) 

In [ ]:
metrics_ex=[
        [loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]
        ,[loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]
        ,[loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]
        ,[loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]
        ,[loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]    
        ,[loss_metrics.MeanAbsoluteRatio(),loss_metrics.AnalogRecall()]
]

load_train_model.compile(
            optimizer=tf.keras.optimizers.Adam(BASE_LEARNING_RATE)                   
            ,loss=loss_metrics.MaskedMeanSquaredError()
            ,loss_weights=[5,5,5,5,1,1]
            ,metrics=metrics_ex                           
            )

In [ ]:
import cv2 as cv
ds_test = dataset_builder.build_validation_ds(tfrecord_files_train[0], model_ds.place_training_labels, cfg);
# img_file = cv.imread("../dataset/images/000000000036.jpg");
train_history_ex = load_train_model.predict(ds_test);

In [ ]:
print(train_history_ex)
